##Ortalama Kayma Takibi

https://stackoverflow.com/questions/4831813/image-segmentation-using-mean-shift-explained

In [ ]:
import numpy as np
import cv2


In [ ]:

# Video akışı yakalama
cap = cv2.VideoCapture(0)

# videonun ilk karesini al
ret,frame = cap.read()


# İlk İzleme Penceresini Ayarlama


# İlk önce yüzü tespit edeceğiz ve bunu başlangıç ​​kutumuz olarak ayarlayacağız.
face_cascade = cv2.CascadeClassifier('../DATA/haarcascades/haarcascade_frontalface_default.xml')
face_rects = face_cascade.detectMultiScale(frame)

# Bu tek dizi listesini (x,y,w,h) dizisine dönüştürün
(face_x,face_y,w,h) = tuple(face_rects[0])
track_window = (face_x,face_y,w,h)
# izleme için yatırım getirisini ayarlama
roi = frame[face_y:face_y+h, face_x:face_x+w]


# HSV Renk Eşlemesini kullanın
hsv_roi =  cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

# Anlamsız hesaplama için her karede hedefi geri yansıtmak üzere histogramı bulun
roi_hist = cv2.calcHist([hsv_roi],[0],None,[180],[0,180])

# Minimum 0 ve maksimum 255 olarak verilen histogram dizisi değerlerini normalleştirin
cv2.normalize(roi_hist,roi_hist,0,255,cv2.NORM_MINMAX)


# Sonlandırma kriterlerini 10 yinelemeyle ayarlayın veya en az 1 puan hareket ettirin
term_crit = ( cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1 )

while True:
    ret ,frame = cap.read()
    if ret == True:

        # HSV'deki Çerçeveyi Yakalayın
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        # Daha önce oluşturulan roi_hist'e dayanarak Geri Projeksiyon'u hesaplayın
        dst = cv2.calcBackProject([hsv],[0],roi_hist,[0,180],1)

        # Dikdörtgenin yeni koordinatlarını elde etmek için anlam kaydırmayı uygulayın
        ret, track_window = cv2.meanShift(dst, track_window, term_crit)

        # Görüntünün üzerine yeni dikdörtgeni çizin
        x,y,w,h = track_window
        img2 = cv2.rectangle(frame, (x,y), (x+w,y+h), (0,0,255),5)

        cv2.imshow('img2',img2)


        k = cv2.waitKey(1) & 0xff
        if k == 27:
            break

    else:
        break

cv2.destroyAllWindows()
cap.release()

## CamShift Takibi

In [ ]:
# Video akışı yakalama
cap = cv2.VideoCapture(0)

# videonun ilk karesini al
ret,frame = cap.read()


# İlk İzleme Penceresini Ayarlama


# İlk önce yüzü tespit edeceğiz ve bunu başlangıç ​​kutumuz olarak ayarlayacağız.
face_cascade = cv2.CascadeClassifier('../DATA/haarcascades/haarcascade_frontalface_default.xml')
face_rects = face_cascade.detectMultiScale(frame)

# Bu tek dizi listesini (x,y,w,h) dizisine dönüştürün
(face_x,face_y,w,h) = tuple(face_rects[0])
track_window = (face_x,face_y,w,h)
# set up the ROI for tracking
roi = frame[face_y:face_y+h, face_x:face_x+w]


# HSV Renk Eşlemesini kullanın
hsv_roi =  cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

# Anlamsız hesaplama için her karede hedefi geri yansıtmak üzere histogramı bulun
roi_hist = cv2.calcHist([hsv_roi],[0],None,[180],[0,180])

#Minimum 0 ve maksimum 255 olarak verilen histogram dizisi değerlerini normalleştirin
cv2.normalize(roi_hist,roi_hist,0,255,cv2.NORM_MINMAX)


# Sonlandırma kriterlerini 10 yinelemeyle ayarlayın veya en az 1 puan hareket ettirin
term_crit = ( cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1 )

while True:
    ret ,frame = cap.read()
    if ret == True:

        # HSV'deki Çerçeveyi Yakalayın
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        # Daha önce oluşturulan roi_hist'e dayanarak Geri Projeksiyon'u hesaplayın
        dst = cv2.calcBackProject([hsv],[0],roi_hist,[0,180],1)

        #########################################################
        #########################################################
        ########## KAM KAYDIRMA ####################################
        ########################################################
        #######################################################

        # Dikdörtgenin yeni koordinatlarını elde etmek için Camshift'i uygulayın
        ret, track_window = cv2.CamShift(dst, track_window, term_crit)

        # Resmin üzerine çiz
        pts = cv2.boxPoints(ret)
        pts = np.int0(pts)
        img2 = cv2.polylines(frame,[pts],True, (0,0,255),5)
        cv2.imshow('img2',img2)

        ########################################################
        #######################################################
        ########################################################
        #######################################################

        k = cv2.waitKey(1) & 0xff
        if k == 27:
            break

    else:
        break

cv2.destroyAllWindows()
cap.release()